# Add the necessary lines to the ini's

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import healpy as hp
import ligo.skymap.plot as ligoPlot
import bilby as bb

mpl.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",  # or "sans-serif" or "monospace"
    "font.serif": ["Computer Modern Roman"],
    "axes.labelsize": 12,
    "font.size": 12,
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10
})

plt.style.use("seaborn-v0_8-paper")

In [2]:
batchPath = ["BBH_Batch0.csv"]

NSBHFilePath = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mockCBCCatalogs/n=1e7,NSBHs,FromSkySim50_withBilbySNRs.csv"
BBHFilePath = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mockCBCCatalogs/n=1e7,BBHs,FromSkySim50_withBilbySNRs.csv"
catalogPath = "catalogList"

In [3]:
def makeLabel(batch,item,spin):
    """
    Label should follow the following structure:
    batch_item_spin
    Where each keyword means the following
    batch: the batch production of the CBC catalog, 
           where all the CBC items are children of
    item: the individual CBC within a batch, usually 
          ranging from 0-300ish
    spin: 0/1, indicating spins misaligned (0), or 
          spins aligned (1)
    """
    return "{}_{}_{}".format(batch,item,spin)

def makeOutDir(base,spec):
    return os.path.join(base,spec)

def makePriorFilePath(typ,spin):
    if typ not in ("BBH","NSBH"):
        raise ValueError("{} is not one of supported types: ('BBH','NSBH')".format(typ))
        return -1
    if spin not in (1,0):
        raise ValueError("{} is not one of supported spins: (0,1)".format(spin))
        return -1

    if typ=="BBH":
        if spin==1: # BBH, spins aligned
            return "/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/aligned_spins_bbh_gwtc3.prior"
        else: # BBH, spins misaligned
            return "/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/precessing_spins_bbh_gwtc3.prior"
    else:
        if spin==1: # NSBH, spins aligned
            return "/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/aligned_spins_nsbh_gwtc3.prior"
        else: # NSBH, spins misaligned
            return "/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/precessing_spins_nsbh_gwtc3.prior"

def selectSpin():
    return np.random.randint(0,2)
    
def makeInjection_dict(injKeys,row):
    myDict = {}
    for k in injKeys:
        if k in ("chi_1", "chi_2"): # handling the tilt/chi issues
            myDict[k] = row["tilt_{}".format(k[-1])]
        elif k in ("ra","dec"):
            myDict[k] = row["m"+k]
        else:
            myDict[k] = row[k]
    return myDict

def getInjection_keys(typ,spin,ppath):
    if typ not in ("BBH","NSBH"):
        raise ValueError("{} is not one of supported types: ('BBH','NSBH')".format(typ))
        return -1
    if spin not in (1,0):
        raise ValueError("{} is not one of supported spins: (0,1)".format(spin))
        return -1
    return bb.gw.prior.CBCPriorDict(ppath).sample().keys()

## Order
- makeLabel
- outDirectory
- prior file path
- injection dictionary

In [4]:
CBCType="BBH"
batch=0

In [6]:
toWrite = np.array([])

for b,batch in zip(batchPath,range(len(batchPath))):
    batchDF = pd.read_csv(os.path.join(os.getcwd(),catalogPath,b))
    basePath = "/pscratch/sd/s/seanmacb/proj411ProdRuns/catalogs/batch_"+str(batch)
    os.makedirs(basePath,exist_ok=True)

    for item,row in batchDF.iterrows():
        fullItem = "{}_{}".format(item,CBCType)
        spin=selectSpin() # Randomly choose spins to be aligned (1) or misaligned (0)
        label = makeLabel(batch,fullItem,spin)
        outDir = makeOutDir(basePath,label)
        priorPath = makePriorFilePath(CBCType,spin)
        injKeys = getInjection_keys(CBCType,spin,priorPath)
        injDict = makeInjection_dict(injKeys,row)

        toWrite = np.append(toWrite,[label,outDir,priorPath,injDict])

# Reshape the final array
toWrite = np.reshape(toWrite,(-1,4))

In [7]:
fieldKeys = ["label","outdir","prior-file", "injection-dict"]
batch0_basepath = "/pscratch/sd/s/seanmacb/proj411ProdRuns/catalogs/batch_0/iniFiles"

In [8]:
for writeables in toWrite:
    with open("BBH_base.ini", "r") as f:
        read = f.readlines()

    writeFile = os.path.join(batch0_basepath,writeables[0])+".ini"
    
    for field,value in zip(fieldKeys,writeables):
        read.append("\n")
        read.append("{}={}\n".format(field,value))

    with open(writeFile, "w") as f:
        f.writelines(read)